In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import seaborn as sns
import os
import os.path as osp
import sys
import pickle
import joblib
from collections import Counter
from itertools import product
import torch
import pdb
import random
import tables
from sklearn.linear_model import LogisticRegression, LinearRegression
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, average_precision_score,\
balanced_accuracy_score
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.calibration import CalibratedClassifierCV
import heckman
from heckman import Heckman
import wandb
from wandb.lightgbm import wandb_callback, log_summary
from dill.source import getsource
from dill import detect
import functools

### set the seeds and change to current directory + set the output directory

In [2]:
SEED=90210
np.random.seed(SEED)
os.environ['USER_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/'
os.environ['OUT_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/output_directory'

### Create function to pickle functions

In [3]:
def function_to_string(fn):
    return getsource(detect.code(fn)) 

In [4]:
processed_data_path=osp.join(os.environ.get('OUT_PATH'), 'stop_and_frisk')

In [5]:
os.environ['WANDB_SILENT']="true"

In [6]:
os.chdir('/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/')
from AnalysisFuncs import trainEvalModel, trainHardPseudo, loadFile, saveFile

In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
train_X = loadFile(osp.join(processed_data_path,'predict_T'), '/train_X.npy')
train_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/train_y_T.npy')
val_X = loadFile(osp.join(processed_data_path,'predict_T'), '/val_X.npy')
val_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/val_y_T.npy')
train_cross_val_X = loadFile(osp.join(processed_data_path,'predict_T'), '/train_cross_val_X.npy')
train_cross_val_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/train_cross_val_y_T.npy')
test_X = loadFile(osp.join(processed_data_path,'predict_T'), '/test_X.npy')
test_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/test_y_T.npy')

train_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/train_X_D_given_T.npy')
train_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/train_y_D_given_T.npy')
val_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/val_X_D_given_T.npy')
val_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/val_y_D_given_T.npy')
train_cross_val_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), 
                                       '/train_cross_val_X_D_given_T.npy')
train_cross_val_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), 
                                       '/train_cross_val_y_D_given_T.npy')
test_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/test_X_D_given_T.npy')
test_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/test_y_D_given_T.npy')



train_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/train_y_D_and_T.npy')
val_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/val_y_D_and_T.npy')
train_cross_val_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), 
                                       '/train_cross_val_y_D_and_T.npy')
test_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/test_y_D_and_T.npy')

In [9]:
df_cleaned = loadFile(processed_data_path, 'df_cleaned.csv')
test_idxs = loadFile(processed_data_path, '/test_idxs.npy')
train_idxs = loadFile(processed_data_path, '/train_idxs.npy')
val_idxs = loadFile(processed_data_path, '/val_idxs.npy')
assert len(np.intersect1d(train_idxs, val_idxs))==0
assert len(np.intersect1d(test_idxs, val_idxs))==0
assert len(np.intersect1d(train_idxs, test_idxs))==0
tr=len(train_idxs)
v=len(val_idxs)
t=len(test_idxs)
print(f"train %:{tr*100/(tr+v+t):.2f}, val %:{v*100/(tr+v+t):.2f}, test %:{t*100/(tr+v+t):.2f}")

/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


train %:45.00, val %:5.00, test %:50.00


In [10]:
# load the best params as pickle if they exist , if not use the default settings
random_state=0
bst_params_T_LR={'random_state': random_state, 
              'solver': 'liblinear', 
              'penalty':  'l1',
                }
bst_params_T_LGBM={'random_state': random_state, 
                  }
LR_T = LogisticRegression(**bst_params_T_LR)
LGBM_T = LGBMClassifier(**bst_params_T_LGBM)

bst_params_D_given_T_LR = {'random_state': random_state, 
                      'solver': 'liblinear', 
                      'penalty':  'l1',
                       }
# bst_params_D_given_T_LGBM={'random_state': random_state, 
#                   }
bst_params_D_given_T_LGBM = {'bagging_fraction': 0.7892603819633417, 'feature_fraction': 0.5283564886587215, 
             'max_depth': 7, 'min_child_samples': 30}
LR_D_given_T = LogisticRegression(**bst_params_D_given_T_LR)
LGBM_D_given_T = LGBMClassifier(**bst_params_D_given_T_LGBM)

bst_params_D_and_T_LR = {'random_state': random_state, 
                      'solver': 'liblinear', 
                      'penalty':  'l1',
                     }
bst_params_D_and_T_LGBM={'random_state': random_state, 
                  }
LR_D_and_T = LogisticRegression(**bst_params_D_and_T_LR)
LGBM_D_and_T = LGBMClassifier(**bst_params_D_and_T_LGBM)

In [12]:
calibrate_method='calibrate_by_bins'
cv=20

In [11]:
model_descr="predicting T using Logistic Regression"
LR_T, test_proba_LR_T = trainEvalModel(LR_T, train_cross_val_X, train_cross_val_y_T, test_X, test_y_T, 
                                       model_descr, calibrate=False, calibrate_method=calibrate_method)

model_descr="predicting D|T using Logistic Regression"
LR_D_given_T, test_proba_LR_D_given_T = trainEvalModel(LR_D_given_T, train_cross_val_X_D_given_T, 
                    train_cross_val_y_D_given_T, test_X_D_given_T, test_y_D_given_T, model_descr, 
                                                       calibrate=False, calibrate_method=calibrate_method)


AUC score :predicting T using Logistic Regression: 0.818
AUPR score :predicting T using Logistic Regression: 0.851
AUC score :predicting D|T using Logistic Regression: 0.802
AUPR score :predicting D|T using Logistic Regression: 0.242


In [13]:
model_descr="predicting T using LGBM"
LGBM_T, test_proba_LGBM_T = trainEvalModel(LGBM_T, train_cross_val_X, train_cross_val_y_T, test_X, test_y_T, 
                                       model_descr, calibrate=False, calibrate_method=calibrate_method, cv=cv)


AUC score :predicting T using LGBM: 0.833
AUPR score :predicting T using LGBM: 0.866


In [14]:
model_descr="predicting D|T using LGBM"
LGBM_D_given_T, test_proba_LGBM_D_given_T = trainEvalModel(LGBM_D_given_T, train_cross_val_X_D_given_T, 
                    train_cross_val_y_D_given_T, test_X_D_given_T, test_y_D_given_T, model_descr, 
                                                           calibrate=False, calibrate_method=calibrate_method, cv=cv)

[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. C

In [14]:
model_descr="predicting D,T using Logistic Regression"
LR_D_and_T, test_proba_LR_D_and_T = trainEvalModel(LR_D_and_T, train_cross_val_X, 
                    train_cross_val_y_D_and_T, test_X, test_y_D_and_T, model_descr, 
                                                   calibrate=False, calibrate_method=calibrate_method)


AUC score :predicting D,T using Logistic Regression: 0.845
AUPR score :predicting D,T using Logistic Regression: 0.210


In [15]:
model_descr="predicting D,T using LGBM"
LGBM_D_and_T, test_proba_LGBM_D_and_T = trainEvalModel(LGBM_D_and_T, train_cross_val_X, 
                    train_cross_val_y_D_and_T, test_X, test_y_D_and_T, model_descr, 
                                                       calibrate=False, calibrate_method=calibrate_method)

AUC score :predicting D,T using LGBM: 0.865
AUPR score :predicting D,T using LGBM: 0.239


In [16]:
def getClippedProbs(model, train_X, epsilon=0.05):
    probs= model.predict_proba(train_X)[:,1]
    probs=np.clip(probs, a_min=epsilon, a_max=1-epsilon)
    return probs

In [26]:
model_descr="predicting D|T_IPW using Logistic Regression"
train_X_D_given_T_probs_LR = getClippedProbs(LR_T, train_cross_val_X_D_given_T)
LR_D_given_T_ipw, test_proba_LR_D_given_T_ipw = trainEvalModel(LR_D_given_T, train_cross_val_X_D_given_T, 
                    train_cross_val_y_D_given_T, test_X_D_given_T, test_y_D_given_T, model_descr,
                                                               calibrate=False, calibrate_method=calibrate_method)


AUC score :predicting D|T_IPW using Logistic Regression: 0.802
AUPR score :predicting D|T_IPW using Logistic Regression: 0.242


In [17]:
model_descr="predicting D|T_IPW using LGBM"
train_X_D_given_T_probs_LGBM = getClippedProbs(LGBM_T, train_cross_val_X_D_given_T)
LGBM_D_given_T_ipw, test_proba_LGBM_D_given_T_ipw = trainEvalModel(LGBM_D_given_T, train_cross_val_X_D_given_T, 
                    train_cross_val_y_D_given_T, test_X_D_given_T, test_y_D_given_T, model_descr,
                                                            calibrate=False, calibrate_method=calibrate_method)

[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. C

In [18]:
#use isotonic regression for calibration in the path to save the model
processed_data_path = osp.join(processed_data_path, calibrate_method)

In [29]:
#save LR models
saveFile(osp.join(processed_data_path,'predict_T'), LR_T, 'LR_T.pkl')

saveFile(osp.join(processed_data_path,'predict_D_given_T'), LR_D_given_T, 'LR_D_given_T.pkl')

saveFile(osp.join(processed_data_path,'predict_D_and_T'), LR_D_and_T, 'LR_D_and_T.pkl')

saveFile(osp.join(processed_data_path,'predict_D_given_T_ipw'), LR_D_given_T_ipw, 'LR_D_given_T_ipw.pkl')

In [19]:
# save LGBM models
saveFile(osp.join(processed_data_path,'predict_T'), LGBM_T, 'LGBM_T.pkl')
saveFile(osp.join(processed_data_path,'predict_D_given_T'), LGBM_D_given_T, 'LGBM_D_given_T.pkl')
saveFile(osp.join(processed_data_path,'predict_D_and_T'), LGBM_D_and_T, 'LGBM_D_and_T.pkl')
saveFile(osp.join(processed_data_path,'predict_D_given_T_ipw'), LGBM_D_given_T_ipw, 'LGBM_D_given_T_ipw.pkl')

In [31]:
random_state=0
bst_params_D_pseudo_LR = bst_params_D_given_T_LR
bst_params_D_pseudo_LGBM = bst_params_D_given_T_LGBM
LR_D_pseudo = LogisticRegression(**bst_params_D_pseudo_LR)
LGBM_D_pseudo = LGBMClassifier(**bst_params_D_pseudo_LGBM)

In [37]:
#predict the outcomes
#should be using copy and deepcopy ?

#assert that the pseudo labels are only binary in nature and not probs
train_cross_val_idxs=np.concatenate((train_idxs, val_idxs))
LR_D_pseudo, test_probs_D_pseudo_LR = trainHardPseudo(LR_D_given_T, df_cleaned, train_idxs, val_idxs,
                        train_X, val_X, LR_D_pseudo, test_X_D_given_T, test_y_D_given_T, 
                        model_descr="Logistic Regression with hard pseudo labels", 
                                                      calibrate=False, calibrate_method=calibrate_method)

AUC score :Logistic Regression with hard pseudo labels: 0.799
AUPR score :Logistic Regression with hard pseudo labels: 0.243


In [38]:
LGBM_D_pseudo, test_probs_D_pseudo_LGBM = trainHardPseudo(LGBM_D_given_T, df_cleaned, train_idxs, val_idxs,
                        train_X, val_X, LGBM_D_pseudo, test_X_D_given_T, test_y_D_given_T, 
                        model_descr="LGBM with hard pseudo labels", 
                                                          calibrate=False, calibrate_method=calibrate_method)

[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7892603819633417
[LightGBM] [Warning] feature_fraction is set=0.5283564886587215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5283564886587215
[LightGBM] [Warning] bagging_fraction is set=0.7892603819633417, subsample=1.0 will be ignored. C

In [45]:
processed_data_path

'/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/output_directory/stop_and_frisk/isotonic'

In [46]:
#save LR models
saveFile(osp.join(processed_data_path,'predict_D_pseudo'), LR_D_pseudo, 'LR_D_pseudo.pkl')


In [47]:
# save LGBM models
saveFile(osp.join(processed_data_path,'predict_D_pseudo'), LGBM_D_pseudo, 'LGBM_D_pseudo.pkl')